In [70]:
import pandas as pd
import numpy as np

import codecs, json

In [71]:
##################### SINGLE FILE #######################

In [72]:
with open('/Users/calmaleh/Desktop/school/project_course/jeppesen/ac_poor_4.bsad') as json_file:
    json_data = json.load(json_file)

frames = []
for j in range(len(json_data['tables'])):
    df = pd.DataFrame(np.array(json_data['tables'][j]['table'])[:,:], 
                           columns = json_data['tables'][j]['header']['variables'])
    df['MODE'] = json_data['tables'][j]['header']['mode']
    df['STATE'] = json_data['tables'][j]['header']['flightphase']
    if df['STATE'][0] == 'cruise':
        frames.append(df)
    
df = pd.concat(frames,ignore_index=True)
df = df[['DISA','ALTITUDE','MASS','MODE','FUELFLOW', 'TAS','STATE']]
df = df[(df.MODE != 'Long Range Cruise') & (df.MODE != 'Max Cruise Thrust') & (df.MODE != 'Max Cruise Thrust') &
       (df.MODE != 'Max Range Cruise') & (df.MODE != 'Max Continuous Thrust Cruise')]


In [73]:
##################### ALL FILES #######################

In [74]:
## Print out every mode for each data file
for i in range(7):
    with open('/Users/calmaleh/Desktop/school/project_course/jeppesen/ac_poor_' + str(i+1) +'.bsad') as json_file:
        json_data = json.load(json_file)
        for i in range(8):
            print(json_data['tables'][i]['header']['mode'])
    print('\n')

300 KIAS/M0.80
1 Engine EROPS
2LRC OXY EROPS
Mach 0.80
Mach 0.83
Mach 0.86
Max Cruise Thrust
Max Range Cruise


250/300 KIAS/M0.78
1LRC EROPS OXY ICE
250 KIAS/M0.77
2LRC EROPS OXY ICE
300 KIAS/M0.80
300 KIAS/M0.82
Long Range Cruise
M0.78/300/250 KIAS


Normal Climb
1MCT EROPS ICE
2LRC EROPS OXY ICE
Long Range Cruise
Mach 0.75
Mach 0.80
Mach 0.84
Max Cruise Thrust


300 KIAS/M0.80
2MCT EROPS
3LRC OXY EROPS
Long Range Cruise
Mach 0.75
Mach 0.80
Max Cruise Thrust
M0.80/300 KIAS


250/300/M0.87
1 LRC EROPS
2 LRC OXY EROPS
Long Range Cruise
Mach 0.85
Mach 0.86
Mach 0.87
Mach 0.88


280 KIAS/M0.76
1ENG EROPS LRC
2ENG EROPS OXY LRC
Mach 0.76
Mach 0.78
Mach 0.80
Mach 0.82
300 KIAS/M0.82


250 KIAS/M0.70
1ENG EROPS M0.45
2ENG EROPS OXY
Long Range Cruise M0.76
Mach 0.78
Mach 0.80
Max Continuous Thrust Cruise
Max Range Cruise M0.74




In [75]:
df = df[df['MODE'].str.contains("Mach")]
df['MODE'] = df['MODE'].map(lambda x: x.lstrip('Mach '))


In [76]:
df

,DISA,ALTITUDE,MASS,MODE,FUELFLOW,TAS,STATE
420,0.0,0.0,21092.045205,0.75,0.176901,250.0,cruise
421,0.0,10000.0,21092.045205,0.75,0.162159,300.0,cruise
422,0.0,14000.0,21092.045205,0.75,0.167829,339.0,cruise
423,0.0,25000.0,21092.045205,0.75,0.183516,449.0,cruise
424,0.0,31000.0,21092.045205,0.75,0.150630,438.0,cruise
...,...,...,...,...,...,...,...
775,10.0,35000.0,10432.624510,0.80,0.125929,467.0,cruise
776,10.0,37000.0,10432.624510,0.80,0.117493,464.0,cruise
777,10.0,45000.0,10432.624510,0.80,0.090385,464.0,cruise
778,10.0,47000.0,10432.624510,0.80,0.086970,464.0,cruise


In [214]:
df = pd.DataFrame([])
for i in range(7):
    with open('/Users/calmaleh/Desktop/school/project_course/jeppesen/ac_poor_' + str(i+1) +'.bsad') as json_file:
        json_data = json.load(json_file)
      
    frames = []
    for j in range(len(json_data['tables'])):
        temp_df = pd.DataFrame(np.array(json_data['tables'][j]['table'])[:,:], 
                               columns = json_data['tables'][j]['header']['variables'])
        temp_df['MODE'] = json_data['tables'][j]['header']['mode']
        temp_df['STATE'] = json_data['tables'][j]['header']['flightphase']
        if temp_df['STATE'][0] == 'cruise':
            frames.append(temp_df)

    temp_df = pd.concat(frames,ignore_index=True)
    temp_df = temp_df[['DISA','ALTITUDE','MASS','MODE','FUELFLOW', 'TAS']]
    temp_df = temp_df[(temp_df.MODE != 'Long Range Cruise') & (temp_df.MODE != 'Max Cruise Thrust')]
    
    df = pd.concat([df,temp_df])



In [216]:
def speed_converter(df):
    TROPOPAUSE_ALT = 11000.0
    STRATOSPHERE_ALT = 20000

    TROPOPAUSE_TEMP = 216.65
    STD_TEMP = 288.15

    STD_LAPSE_RATE = -0.0065

    SPEED_OF_SOUND = 340.294

    df['ISA_CONDITION'] = False
    df.loc[(0 <= df['ALTITUDE']) & (df['ALTITUDE'] <= TROPOPAUSE_ALT), 'ISA_CONDITION'] = True

    df['ISA'] = 0

    df.loc[df.ISA_CONDITION == False, 'ISA'] = TROPOPAUSE_TEMP
    df.loc[df.ISA_CONDITION == True, 'ISA'] = round(STD_TEMP + STD_LAPSE_RATE * df[df.ISA_CONDITION == True].ALTITUDE,2)
    df['SPEED_SOUND'] = SPEED_OF_SOUND * np.sqrt((df['ISA'] + df['DISA'])/ STD_TEMP)

    df['MACH'] = round(df['TAS'] / df['SPEED_SOUND'],4)
    df[df.MACH==2]
    df.drop(['ISA_CONDITION', 'ISA','SPEED_SOUND'],axis=1)

,DISA,ALTITUDE,MASS,MODE,FUELFLOW,TAS,MACH
0,0.0,0.0,16193.247609,1 Engine EROPS,0.111067,249.0,0.7317
1,0.0,10000.0,16193.247609,1 Engine EROPS,0.107854,267.0,0.8916
2,0.0,19000.0,16193.247609,1 Engine EROPS,0.107854,267.0,0.9049
3,0.0,27000.0,16193.247609,1 Engine EROPS,0.107854,267.0,0.9049
4,0.0,33000.0,16193.247609,1 Engine EROPS,0.107854,267.0,0.9049
...,...,...,...,...,...,...,...
2131,20.0,41000.0,4082.331330,Max Range Cruise M0.74,0.044162,444.0,1.4397
2132,20.0,43000.0,4082.331330,Max Range Cruise M0.74,0.041138,442.0,1.4333
2133,20.0,45000.0,4082.331330,Max Range Cruise M0.74,0.036665,423.0,1.3716
2134,20.0,47000.0,4082.331330,Max Range Cruise M0.74,0.031625,388.0,1.2582
